<div style="padding:20px;color:white;margin:0;font-size:200%;text-align:center;display:fill;border-radius:5px;background-color:#46BAED;overflow:hidden;font-weight:bold">TPS April 2022</div>

# <span style="color:#46BAED;margin:0;text-align:left;font-weight:bold">Competition Objective</span>
This month's [TPS Competition](https://www.kaggle.com/competitions/tabular-playground-series-apr-2022) is a time series classification problem. The goal of this competition is to identify whether a subject is in Activity State 0 or 1 based on 60-second sequences of biological sensor data recorded from several hundred participants.

# <span style="color:#46BAED;margin:0;text-align:left;font-weight:bold">Data Overview</span>
The training set comprises ~26,000 60-second recordings of thirteen biological sensors for almost one thousand experimental participants. The variables in the dataset include:
- `sequence`: a unique ID for each sequence.
- `subject`: a unique ID for each subject in the experiment.
- `step`: the time step of the recording, in one second intervals.
- `sensor_00` - `sensor_12`: the value for each of the thirteen sensors at that time step.

The train_labels.csv contains:
- `sequence`: the unique ID for each sequence.
- `state`: the state associated to each sequence, our target variable.

The final predictions for each participant's activity state will be made based on the sensor information for ~12,000 sequences in the test set.

In [ ]:
import warnings, gc
import numpy as np 
import pandas as pd
import matplotlib as mpl
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
from scipy.stats import gaussian_kde
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import KFold, GroupKFold 
from sklearn.metrics import roc_auc_score, roc_curve, auc
from lightgbm import LGBMClassifier

warnings.filterwarnings("ignore")
init_notebook_mode(connected=True)
color=px.colors.qualitative.Plotly
temp=dict(layout=go.Layout(font=dict(family="Franklin Gothic", size=12), 
                           height=500, width=700))

train=pd.read_csv('../input/tabular-playground-series-apr-2022/train.csv')
test=pd.read_csv('../input/tabular-playground-series-apr-2022/test.csv')
sub=pd.read_csv('../input/tabular-playground-series-apr-2022/sample_submission.csv')
lbl=pd.read_csv('../input/tabular-playground-series-apr-2022/train_labels.csv')

print("Train Shape: There are {:,.0f} rows and {:,.0f} columns.\nMissing values = {}, Duplicates = {}.\n".
      format(train.shape[0], train.shape[1],train.isna().sum().sum(), train.duplicated().sum()))
print("There are {:,.0f} training labels.\n".format(lbl.shape[0]))
print("Test Shape: There are {:,.0f} rows and {:,.0f} columns.\nMissing values = {}, Duplicates = {}.\n".
      format(test.shape[0], test.shape[1], test.isna().sum().sum(), test.duplicated().sum()))

train=train.merge(lbl, on='sequence')
df0=train[train.state==0].describe().reset_index()
df1=train[train.state==1].describe().reset_index()
df0['state']=0
df1['state']=1
df=pd.concat([df0,df1], axis=0).set_index(['index','state'])
df.index = df.index.set_names(['','State'])
df=df.reindex([('count', 0),('count', 1),( 'mean', 0),( 'mean', 1),
               (  'std', 0), (  'std', 1),(  'min', 0),(  'min', 1),
               (  '25%', 0), (  '25%', 1),(  '50%', 0),(  '50%', 1),
               (  '75%', 0),(  '75%', 1),(  'max', 0),(  'max', 1)])
display(df.style.format('{:,.2f}')
        .background_gradient(subset=(df.index[2:],df.columns[:]), cmap='GnBu', axis=0))

# <b><span style="color:#46BAED;margin:0;text-align:left">Exploratory Data Analysis</span></b>

In [ ]:
p1=pd.DataFrame.from_dict({'Train': train.subject.nunique(), 'Test': test.subject.nunique()}, 
                          orient='index', columns=['count'])
p2=pd.DataFrame.from_dict({'Train': train.sequence.nunique(), 'Test': test.sequence.nunique()}, 
                          orient='index', columns=['count'])
p3=pd.DataFrame.from_dict({'Train':(train.groupby('subject')['step'].count().sum()/train.subject.nunique())/60, 
                           'Test':(test.groupby('subject')['step'].count().sum()/test.subject.nunique())/60}, 
                          orient='index', columns=['count'])
col_list=['#0969AD','#F2BB08']
rgb=['rgba'+str(mpl.colors.to_rgba(i, 0.55)) for i in col_list]
titles=['Total Participants', '60-Second Sequences',
        'Average Time per Participant']
fig = make_subplots(rows=1, cols=3, 
                    subplot_titles=titles)
text=['','',' min']
legend=True
for i, df in enumerate([p1,p2,p3]):
    if i!=0: legend=False
    for j in range(0,2):
        fig.add_trace(go.Bar(x=[df.index[j]], y=df.iloc[j,:], 
                             text=df.iloc[j,:], name=df.index[j], width=0.8,
                             texttemplate='%{text:,.0f}'+str(text[i]), textposition='outside',
                             marker=dict(color=rgb[j], line_color=col_list[j], line_width=2),
                             hovertemplate='%{x} '+str(titles[i])+' = %{y:,.0f}'+str(text[i]),
                             showlegend=legend),row=1,col=i+1)
fig.update_layout(template=temp, title='Train and Test Set Overview', barmode='group',
                  yaxis1_range=(0,730),yaxis2_range=(0,28000),yaxis3_range=(0,42),
                  legend=dict(orientation="h",yanchor="bottom",xanchor="right",
                              y=1.14,x=.96,traceorder='normal'))
fig.show()

There are 672 participants in the training set with just under 26,000 unique sequences. In the test set, we have 319 participants with about 12,000 sequences. The average length of time for each participant’s sensor data is around 39 minutes in both sets.

In [ ]:
rgb=['rgba'+str(mpl.colors.to_rgba(i, 0.5)) for i in color[:2]]
fig = make_subplots(rows=2, cols=2,
                    specs=[[{},{}],
                           [{"colspan":2}, None]],
                    vertical_spacing=0.18,
                    column_widths=(.25,.75),
                    row_heights=(.55,.45),
                    subplot_titles=('Target Distribution', 
                                    'Unique Sequences per Participant', 
                                    'Proportion of time spent in each state'))

plot1=train.state.value_counts(normalize=True).mul(100).reset_index()
plot1['index']=plot1['index'].apply(lambda x: 'State 0' if x==0 else 'State 1')
fig.add_trace(go.Bar(x=plot1['index'], y=plot1['state'], text=plot1['state'],
                     texttemplate='%{text:.1f}%', textposition='outside',
                     marker=dict(color=rgb[::-1], line_color=color[:2][::-1], 
                                 line_width=2),
                     hovertemplate='%{x} = %{y:.1f}%<extra></extra>',
                     showlegend=False),row=1,col=1)

plot2=train.groupby(['subject','state'])['sequence'].nunique().reset_index()
for i in reversed(range(0,2)): 
    x=plot2[plot2.state==i]['subject']
    y=plot2[plot2.state==i]['sequence']
    fig.add_trace(go.Scatter(x=x, y=y, name='State {}'.format(i), mode='lines', 
                             marker_color=color[i],showlegend=False), 
                  row=1,col=2)

plot3=train.groupby(['subject','state'])['sequence'].nunique()
plot3=plot3.div(plot3.sum(level=0), level=0).mul(100).reset_index()
p0=plot3[plot3.state==0].sort_values(by='sequence').reset_index(drop=True)
p1=plot3[plot3.state==1].sort_values(by='sequence',ascending=False).reset_index(drop=True)
fill=['tozeroy','tonexty']
for i, df in enumerate([p1,p0]):
    fig.add_trace(go.Scatter(x=df.index, y=df.sequence, fill=fill[i], 
                             stackgroup='one',name='State {}'.format(df.state.max()),
                             marker_color=color[:2][::-1][i]),row=2,col=1)
fig.update_xaxes(showline=False, zeroline=False)
fig.update_layout(template=temp, height=800, hovermode='x unified',
                  yaxis1=dict(range=(0,58),ticksuffix='%'),
                  xaxis2_title='Participant',
                  yaxis3_ticksuffix='%', xaxis3_title="Participant",
                  legend=dict(orientation="v",yanchor="bottom",xanchor="right",
                              y=.36,x=1,traceorder='normal'))
fig.show()

Our target variable is evenly balanced with about 50% of sequences in each state. When we look at the number of sequences for each participant, the total amount in State 0 tends to stay under 50, while some participants are in State 1 for more than 100 sequences. Interestingly, we also have about 60 participants who never go into State 1. 

In [ ]:
sensors=[col for col in train if col.startswith('sensor')]
rowtitles=['Sensor {}'.format(sensors[i][-2:]) for i in range(len(sensors))]
t1=['Distribution of Sensor {}'.format(sensors[i][-2:]) for i in range(len(sensors))]
t2=['Median Signals by Second' for i in range(len(sensors))]
titles=[x for y in zip(t1, t2) for x in y]

fig = make_subplots(rows=13, cols=2, 
                    horizontal_spacing=0.1, 
                    subplot_titles=titles, 
                    row_titles=rowtitles)

legend=True
for i, sensor in enumerate(sensors):
    if i != 0:  
        legend=False
    for j in range(0,2):
        hist_data=train[train.state==j][sensor]
        density=gaussian_kde(dataset=hist_data)
        x=np.arange(hist_data.min(), hist_data.max()) 
        density.covariance_factor = lambda: 4  
        density._compute_covariance()
        kde_curve=density(x)
        fig.append_trace(go.Scatter(x=x, y=kde_curve, marker_color=color[j], 
                                    fill='tozeroy', name='State {}'.format(j), 
                                    hovertemplate='Density = %{y:.5f} at %{x}',
                                    showlegend=legend), 
                         row=i+1, col=1)
        fig.update_yaxes(title="Density",row=i+1, col=1)
        
    plot_df=train.groupby(['step','state'])[sensor].median().reset_index()
    for j in reversed(range(0,2)): 
        y=plot_df[plot_df.state==j][sensor]
        x=plot_df.step.unique()
        if y.sum() != 0:
            fig.append_trace(go.Bar(x=x, y=y, name='State {}'.format(j),
                                    marker_color=color[j], 
                                    hovertemplate='Median Signal = %{y:.5f} at second %{x}',
                                    opacity=0.9, showlegend=False),
                             row=i+1, col=2)
            fig.update_xaxes(title='Time, in seconds',row=i+1, col=2)
            fig.update_yaxes(title="Signal",row=i+1, col=2)
        else:
            fig.append_trace(go.Scatter(x=x, y=y, name='State {}'.format(j),
                                        marker_color=color[j], 
                                        showlegend=False),
                             row=i+1, col=2) 
            fig.update_xaxes(title='Time, in seconds',row=i+1, col=2)
            fig.update_yaxes(title="Signal",row=i+1, col=2)
fig.update_layout(template=temp, title='Sensor Signal Distributions', 
                  barmode='relative', height=4000, 
                  legend=dict(orientation="v",yanchor="bottom",
                              xanchor="right",y=1.01,x=.99,
                              traceorder='reversed'))
fig.show()

In both activity states, the majority of the sensors have long, heavy-tailed distributions with a peak centered at 0, except Sensors 4 and 12, which more closely resemble a Gaussian distribution, and Sensor 2, which has a left-skewed distribution. In the graphs of the median values at each second, Sensor 2's median signal is 0 across all 60 time steps for both states. This also occurs in Sensor 8, while the rest of the sensors show State 1 tends to have greater median values than State 0 in both positive and negative directions at each time step. 

In [ ]:
sensors=[col for col in train if col.startswith('sensor')]
corr=train.iloc[:,3:-1].corr(method='spearman')
t=train[train.subject==1]
x,y=[-3,0,0,3],[-1.5,0.25,0,1.5]
xbins=dict(start=-3, end=3, size=0.2)
ybins=dict(start=-1.5, end=1.5, size=0.2)
ax=dict(zeroline=False, showgrid=False,
        mirror=True,showline=True,linecolor='#F1F1F1')
cmap=mpl.colors.LinearSegmentedColormap.from_list("", 
                                                  ["#2290AC", '#B4DAEA', '#FFB3B3',"#E92D22"])
col_scale=[[0, 'white'], [0.25, 'rgb(10,136,186)'], 
           [0.5, 'rgb(242,211,56)'],[0.75, 'rgb(242,143,56)'],
           [1, 'rgb(217,30,30)']]

fig = make_subplots(rows=13, cols=13, 
                    shared_yaxes=True)
# Density plots
for i, sensor in enumerate(sensors):
    x_hist=t[sensor]
    for j, sen in enumerate(sensors[i:]):
        fig.add_trace(go.Histogram2d(x=x_hist, y=t[sen], histnorm='', zsmooth='fast',
                                     xbins=xbins,ybins=ybins, name='', showscale=False,
                                     colorscale=col_scale), row=i+j+1,col=i+1)
        fig.update_yaxes(ax,row=i+j+1,col=i+1)
        fig.update_xaxes(ax,row=i+j+1,col=i+1)

# Correlations
for i, sen in enumerate(sensors):
    c=corr[sen][i+1:]
    if len(c) > 1:
        norm=mpl.colors.TwoSlopeNorm(vmin=corr.min().min()+.1, vcenter=0, vmax=corr.max().max()-.3)
    sensor_cmap=cmap(norm(c)).tolist()
    m=[mpl.colors.rgb2hex(sensor_cmap[rgb]) for rgb in range(len(sensor_cmap))]
    n=i+2
    for j in np.arange(i+2,14):
        fig.add_trace(go.Scatter(x=x,y=y,mode="text",text=["","Corr:", "<br>{:.3f}".format(c[j-n]), ""],
                                 hovertemplate='Sensor {} and {} Correlation = {:.4f}'.format(i,j-n+i+1,c[j-n]),
                                 name='',textfont_color=["#ffffff","#555555","{}".format(m[j-n])],
                                 showlegend=False), 
                      row=i+1,col=j)
        fig.update_yaxes(ax,showticklabels=False, row=i+1,col=j)
        fig.update_xaxes(ax,showticklabels=False, row=i+1,col=j)
fig.update_layout(template=temp, height=1950, width=1000,
                  title='Sensor Correlations and Density Plots', 
                  yaxis1_title='Sensor 0', yaxis14_title='Sensor 1',
                  yaxis27_title='Sensor 2', yaxis40_title='Sensor 3', 
                  yaxis53_title='Sensor 4', yaxis66_title='Sensor 5',
                  yaxis79_title='Sensor 6', yaxis92_title='Sensor 7',
                  yaxis105_title='Sensor 8', yaxis118_title='Sensor 9',
                  yaxis131_title='Sensor 10', yaxis144_title='Sensor 11',
                  yaxis157_title='Sensor 12', xaxis157_title='Sensor 0',
                  xaxis158_title='Sensor 1', xaxis159_title='Sensor 2',
                  xaxis160_title='Sensor 3', xaxis161_title='Sensor 4',
                  xaxis162_title='Sensor 5', xaxis163_title='Sensor 6',
                  xaxis164_title='Sensor 7', xaxis165_title='Sensor 8',
                  xaxis166_title='Sensor 9', xaxis167_title='Sensor 10',
                  xaxis168_title='Sensor 11', xaxis169_title='Sensor 12')
fig.show()

Taking a closer look at the relationships between the Sensors, the highest positive correlations exist between Sensors 3 and 7 and Sensors 0 and 6 of 0.759 and 0.752, respectively. The density plots also display a strong linear relationship between these sensors. Additionally, there is a moderately strong association between Sensor 0 and 9 of 0.612 and Sensor 3 and 11 of 0.57, while Sensors 2, 8, and 12 are the least correlated with the rest of the sensors.

# <b><span style="color:#46BAED;margin:0;text-align:left">Feature Engineering</span></b>
To add additional information to the models, I will include lagged variables and rolling averages for each sensor and take the first difference between each time step. I will also add aggregated sensor information for each sequence and subject, including variables for the mean, median, standard deviation, skewness, kurtosis, minimum, and maximum. These variables will be used in the Logistic Regression and Gradient Boosting models.
The graphs below show the average observed signals and the first differenced signals at each time step for Participant 1. 

In [ ]:
def feat_eng(df):
    
    seq_df=pd.DataFrame()
    sensors=[col for col in df if col.startswith('sensor')]
    
    for sensor in sensors:
        df['{}_lag1'.format(sensor)] = df.groupby('sequence')[sensor].shift(1)
        df['{}_lag1'.format(sensor)].fillna(df[sensor].median(), inplace=True)
        df['{}_diff'.format(sensor)] = df[sensor] - df['{}_lag1'.format(sensor)] 
        df['{}_roll_mean3'.format(sensor)]=df['{}'.format(sensor)].rolling(window=3).mean()
        df['{}_roll_mean6'.format(sensor)]=df['{}'.format(sensor)].rolling(window=6).mean()
        df['{}_roll_mean9'.format(sensor)]=df['{}'.format(sensor)].rolling(window=9).mean()
        df['{}_roll_mean3'.format(sensor)].fillna(df['{}_roll_mean3'.format(sensor)].median(), inplace=True)
        df['{}_roll_mean6'.format(sensor)].fillna(df['{}_roll_mean6'.format(sensor)].median(), inplace=True)
        df['{}_roll_mean9'.format(sensor)].fillna(df['{}_roll_mean9'.format(sensor)].median(), inplace=True)
        s_diff='{}_diff'.format(sensor)
        seq_df['{}_mean'.format(sensor)] = df.groupby(['sequence','subject'])[sensor].mean()
        seq_df['{}_diff_mean'.format(sensor)] = df.groupby(['sequence','subject'])[s_diff].mean()
        seq_df['{}_med'.format(sensor)] = df.groupby(['sequence','subject'])[sensor].median()
        seq_df['{}_std'.format(sensor)] = df.groupby(['sequence','subject'])[sensor].std()
        seq_df['{}_skew'.format(sensor)] = df.groupby(['sequence','subject'])[sensor].skew()
        seq_df['{}_kurt'.format(sensor)] = df.groupby(['sequence','subject'])[sensor].apply(pd.DataFrame.kurt)
        seq_df['{}_min'.format(sensor)] = df.groupby(['sequence','subject'])[sensor].min()
        seq_df['{}_max'.format(sensor)] = df.groupby(['sequence','subject'])[sensor].max()
    
    return df, seq_df.reset_index()

train, seq_df=feat_eng(df=train)
test, test_seq_df=feat_eng(df=test)

In [ ]:
t1=['Sensor {}  Observed Signals'.format(sensors[i][-2:]) for i in range(len(sensors))]
t2=['Differenced Signals' for i in range(len(sensors))]
titles=[x for y in zip(t1, t2) for x in y]

fig = make_subplots(rows=13, cols=2, shared_yaxes=True,
                    horizontal_spacing=0.05, vertical_spacing=0.04,
                    subplot_titles=titles)

t=train[train.subject==1]
for i,sensor in enumerate(sensors): 
    x=np.arange(0,t.sequence.nunique())
    y=t.groupby('sequence')[sensor].mean()
    fig.add_trace(go.Scatter(x=x, y=y, name='Average Signal', 
                             mode='lines+markers',
                             hovertemplate='%{y:.3f} at %{x} minutes',
                             marker=dict(color='#3698CC', size=4), 
                             opacity=0.9, showlegend=False),
                  row=i+1, col=1)
    fig.update_yaxes(title='Signal'.format(i), row=i+1, col=1)
    
sensor_diff=[col for col in t.columns if 'diff' in col]
for i,sensor in enumerate(sensor_diff):
    x=np.arange(0,t.sequence.nunique())
    y=t.groupby('sequence')[sensor].mean()
    fig.add_trace(go.Scatter(x=x, y=y, name='Differenced Signal', 
                             mode='lines+markers', 
                             hovertemplate='%{y:.3f} at %{x} minutes',
                             marker=dict(color='#EE938D', size=4), 
                             showlegend=False),
                  row=i+1, col=2)
fig.update_xaxes(title='Time, in minutes')
fig.update_layout(template=temp, title='Original vs. Differenced Sensor Signals', 
                  hovermode="x unified", height=3500, 
                  legend=dict(orientation="v",yanchor="bottom",xanchor="right",y=1.01,x=.97))
fig.show()

# <b><span style="color:#46BAED;margin:0;text-align:left">Logistic Regression</span></b>

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Concatenate, GRU
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.layers import BatchNormalization, GlobalMaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import metrics, regularizers

tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
colors=px.colors.qualitative.Vivid
def plot_roc(y_val, y_prob):
    fig=make_subplots(rows=1, cols=3)
    for i in range(len(y_val)):
        y=y_val[i]
        prob=y_prob[i]
        fpr, tpr, thresh = roc_curve(y, prob)
        roc_auc = auc(fpr,tpr)
        fig.append_trace(go.Scatter(x=fpr, y=tpr, line=dict(color=colors[i+1], width=3), 
                                    hovertemplate = 'True positive rate = %{y:.3f}, False positive rate = %{x:.3f}',
                                    name='Fold {} AUC = {:.3f}  '.format(i+1,roc_auc)), 
                         row=1,col=i+1)
        fig.add_shape(type="line", xref="x", yref="y", x0=0, y0=0, x1=1, y1=1, 
                      line=dict(color="Black", width=1, dash="dot"), row=1, col=i+1)
        fig.update_xaxes(title='False Positive Rate')
    fig.update_layout(template=temp, title="Cross-Validation ROC Curves", 
                      hovermode="x unified", height=400,
                      yaxis_title='True Positive Rate (Sensitivity)', 
                      legend=dict(orientation='h',y=1.175, x=.5, xanchor="center",
                                  bordercolor="black", borderwidth=.5, font=dict(size=12)))
    fig.show()
    
def plot_predictions(df, title):
    plot_df=pd.DataFrame.from_dict({'State 1':(len(df[df.state>0.5])/len(df.state))*100, 
         'State 0':(len(df[df.state<=0.5])/len(df.state))*100}, orient='index', columns=['pct'])
    fig=go.Figure()
    col=['#F7AA9D','#B1B6FC']
    text=['State 1', 'State 0']
    fig.add_trace(go.Pie(labels=plot_df.index, values=plot_df['pct'], hole=.38, 
                         text=text, showlegend=False,
                         marker=dict(colors=col,line=dict(color=color[:2][::-1],width=2)),
                         hovertemplate = "%{label}: %{value:.2f}%<extra></extra>"))
    fig.update_layout(template=temp, title=title,
                      uniformtext_minsize=15, uniformtext_mode='hide')
    fig.show()

In [ ]:
k_fold = GroupKFold(n_splits=3)
scaler = StandardScaler()

train_seq_df=seq_df.merge(lbl, on='sequence')
y=train_seq_df[['state']]
X=train_seq_df.drop(['sequence','subject','state'], axis=1)
X=pd.DataFrame(scaler.fit_transform(X))
X_test=test_seq_df.drop(['sequence','subject'], axis=1)
X_test=pd.DataFrame(scaler.transform(X_test))

y_valid=[]
glm_preds=[]
test_preds=[]

for fold, (train_index, val_index) in enumerate(k_fold.split(X, y, groups=train_seq_df.subject)):
    
    print("\nFold {}".format(fold+1))
    
    X_train, y_train = X.iloc[train_index,:], y.iloc[train_index,:].values
    X_val, y_val = X.iloc[val_index,:], y.iloc[val_index,:].values
    print("Train shape: {}, {}, Valid shape: {}, {}".format(
        X_train.shape, y_train.shape, X_val.shape, y_val.shape))
        
    with tpu_strategy.scope():
        
        # Logistic Regression
        mod = Sequential()
        mod.add(Dense(1, activation='sigmoid', 
                      kernel_regularizer=regularizers.L2(1e-4), 
                      input_dim=X_train.shape[1]))

        mod.compile(optimizer='adam', loss='binary_crossentropy', 
                    metrics=[metrics.AUC(name = 'auc')])

        mod.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val), verbose=False)

        glm_pred = mod.predict(X_val).squeeze()
        auc_score=roc_auc_score(y_val, glm_pred)
        print("Validation AUC = {:.4f}".format(auc_score))

        y_valid.append(y_val)
        glm_preds.append(glm_pred)
        test_preds.append(mod.predict(X_test).squeeze())
    
    del X_train, y_train, X_val, y_val
    gc.collect()
    
plot_roc(y_val=y_valid, y_prob=glm_preds)    

The Logistic Regression model has an Area Under the Curve (AUC) ranging from 0.78 to 0.8 in each fold.

In [ ]:
sub_glm=sub.copy()
sub_glm['state']=np.mean(test_preds, axis=0)
sub_glm.to_csv("submission_glm.csv", index=False)
plot_predictions(df=sub_glm, title="Predicted Target Distribution,<br>Logistic Regression")

# <b><span style="color:#46BAED">Gradient Boosting</span></b>

In [ ]:
train_seq_df=seq_df.merge(lbl, on='sequence')
y=train_seq_df[['state']]
X=train_seq_df.drop(['sequence','subject','state'], axis=1)
X=pd.DataFrame(scaler.fit_transform(X))
X_test=test_seq_df.drop(['sequence','subject'], axis=1)
X_test=pd.DataFrame(scaler.transform(X_test))

y_valid=[]
gbm_probs=[]
test_preds=[]    
                    
for fold, (train_index, val_index) in enumerate(k_fold.split(X, y, groups=train_seq_df.subject)):
    
    print("\nFold {}".format(fold+1))
    
    X_train, y_train = X.iloc[train_index,:], y.iloc[train_index,:].values
    X_val, y_val = X.iloc[val_index,:], y.iloc[val_index,:].values
    print("Train shape: {}, {}, Valid shape: {}, {}".format(
        X_train.shape, y_train.shape, X_val.shape, y_val.shape))
    
    params = {'boosting_type': 'gbdt',
              'n_estimators': 500,
              'objective': 'binary',
              'learning_rate': 0.1,
              'colsample_bytree': 0.75,
              'subsample': 0.75,
              'metric': 'auc',
              'random_state': 21}
    
    gbm = LGBMClassifier(**params).fit(X_train, y_train, 
                                       eval_set=[(X_train, y_train), (X_val, y_val)],
                                       verbose=150,
                                       eval_metric=['binary_logloss','auc'])
    gbm_prob = gbm.predict_proba(X_val)[:,1]
    y_valid.append(y_val)
    gbm_probs.append(gbm_prob)
    auc_score=roc_auc_score(y_val, gbm_prob)
    print("Validation AUC = {:.4f}".format(auc_score))
    test_preds.append(gbm.predict_proba(X_test)[:,1])
      
    del X_train, y_train, X_val, y_val
    gc.collect()  
    
plot_roc(y_val=y_valid, y_prob=gbm_probs)   

The Gradient Boosting model improved the AUC to over 0.89 on the validation sets.

In [ ]:
sub_gbm=sub.copy()
sub_gbm['state']=np.mean(test_preds, axis=0)
sub_gbm.to_csv("submission_gbm.csv", index=False)
plot_predictions(df=sub_gbm, title='Predicted Target Distribution<br>with Gradient Boosting')

# <b><span style="color:#46BAED;margin:0;font-size:110%;text-align:left">Bidirectional LSTM</span></b>
The last model I will test is a Bidirectional Long Short-Term Memory (LSTM) Network using the sensor information from all 60-second sequences. Bidirectional LSTMs are a type of Recurrent Neural Network that are trained on both directions of input sequences, from past to present (forward) and from present to past (backward). The LSTM model architecture is shown below.

In [ ]:
y=np.array(lbl.state)
X=train.drop(['sequence','subject','step','state'], axis=1)
X=scaler.fit_transform(X)
X=X.reshape(-1, 60, X.shape[1])
X_test=test.drop(['sequence','subject','step'], axis=1)
X_test=scaler.transform(X_test)
X_test=X_test.reshape(-1, 60, X_test.shape[1])
groups=train.sequence.unique()

y_valid=[]
lstm_preds=[]
test_preds=[]    
                    
for fold, (train_index, val_index) in enumerate(k_fold.split(X, y, groups=groups)):
    
    X_train, y_train = X[train_index], y[train_index]
    X_val, y_val = X[val_index], y[val_index]
    
    with tpu_strategy.scope():
        
        x_input = Input(shape=(X.shape[-2:]))
        x1 = Bidirectional(LSTM(units=512, return_sequences=True))(x_input)
        
        l1 = Bidirectional(LSTM(units=384, return_sequences=True))(x1)
        l2 = Bidirectional(LSTM(units=384, return_sequences=True))(x_input)

        c1 = Concatenate(axis=2)([l1,l2])

        l3 = Bidirectional(LSTM(units=256, return_sequences=True))(c1)
        l4 = Bidirectional(LSTM(units=256, return_sequences=True))(l2)
        
        c2 = Concatenate(axis=2)([l3,l4])
        
        l6 = GlobalMaxPooling1D()(c2)
        l7 = Dense(units=128, activation='selu')(l6)
        l8 = Dropout(0.05)(l7)
        
        output = Dense(1, activation='sigmoid')(l8)
        
        lr = ReduceLROnPlateau(monitor='val_auc', factor=0.4,  patience=3, verbose=True)
        es = EarlyStopping(monitor='val_auc', mode='max', patience=5, 
                           restore_best_weights=True, verbose=True)
        
        model = Model(inputs=x_input, outputs=output, 
                      name='Bidirectional_LSTM')
        
        model.compile(optimizer='adam', 
                      loss='binary_crossentropy', 
                      metrics=[metrics.AUC(name = 'auc')])
        
        if fold==0:
            model.summary() 
        
        print("\nFold {}".format(fold+1))
        print("Train shape: {}, {}, Valid shape: {}, {}".format(
            X_train.shape, y_train.shape, X_val.shape, y_val.shape))
        
        model.fit(X_train, y_train,
                  validation_data=(X_val, y_val), epochs=5, batch_size=128, 
                  callbacks=[es,lr], verbose=True)
    
        lstm_pred = model.predict(X_val).squeeze()
        y_valid.append(y_val)
        lstm_preds.append(lstm_pred)
        test_preds.append(model.predict(X_test).squeeze())
      
    del X_train, y_train, X_val, y_val
    gc.collect()

plot_roc(y_val=y_valid, y_prob=lstm_preds) 

Out of the three models, the Bidirectional LSTM had the best performance with a cross-validation Area Under the Curve of over 0.96 in each fold.

In [ ]:
sub_lstm=sub.copy()
sub_lstm['state']=np.mean(test_preds, axis=0)
sub_lstm.to_csv("submission_lstm.csv", index=False)
plot_predictions(df=sub_lstm, title='LSTM Predicted Target Distribution')

### <div style="color:#46BAED;margin:0;font-size:100%;text-align:center">Thank you for reading!</div>